# José Luis Padilla 
## Modulo 3 - Caso práctico 2.1: Calidad de datos (conjunto de datos de pasajeros del Titanic)

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets

## Descripción del caso 

El hundimiento del Titanic es uno de los naufragios más infames de la historia. El 15 de abril de 1912, durante su viaje inaugural, el Titanic se hundió después de chocar con un iceberg, matando a miles de personas. Esta tragedia sensacional conmocionó a la comunidad internacional y condujo a mejores normas de seguridad aplicables a los buques. Una de las razones por las que el naufragio dio lugar a semejante cantidad de muertes fue que no había suficientes botes salvavidas para los pasajeros y la tripulación. Aunque hubo algún elemento de suerte involucrada en sobrevivir al hundimiento, algunos grupos de personas tenían más probabilidades de sobrevivir que otros, como las mujeres, los niños y la clase alta.

En este caso práctico se demuestran algunas técnicas básicas de exploración de datos, "profiling" de la calidad de datos y mitigación de problemas de calidad de datos para conseguir fines prácticos. Como datos para ilustrar las técnicas se utiliza el conjunto de datos de pasajeros del Titanic (al que se accede a través de la librería sckit-learn, de modo que el código pueda ejecutarse en cualquier entorno sin más dependencias)

## Acceso y exploración de los datos 

### Descripción de los datos (obtenida ejecutando el código comentado en la celda anterior)

**Author**: Frank E. Harrell Jr., Thomas Cason  
**Source**: [Vanderbilt Biostatistics](http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.html)  
**Please cite**:   

The original Titanic dataset, describing the survival status of individual passengers on the Titanic. The titanic data does not contain information from the crew, but it does contain actual ages of half of the passengers. The principal source for data about Titanic passengers is the Encyclopedia Titanica. The datasets used here were begun by a variety of researchers. One of the original sources is Eaton & Haas (1994) Titanic: Triumph and Tragedy, Patrick Stephens Ltd, which includes a passenger list created by many researchers and edited by Michael A. Findlay.

Thomas Cason of UVa has greatly updated and improved the titanic data frame using the Encyclopedia Titanica and created the dataset here. Some duplicate passengers have been dropped, many errors corrected, many missing ages filled in, and new variables created. 

For more information about how this dataset was constructed:
http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3info.txt


**Attribute information**

The variables on our extracted dataset are pclass, survived, name, age, embarked, home.dest, room, ticket, boat, and sex. pclass refers to passenger class (1st, 2nd, 3rd), and is a proxy for socio-economic class. Age is in years, and some infants had fractional values. The titanic2 data frame has no missing data and includes records for the crew, but age is dichotomized at adult vs. child. These data were obtained from Robert Dawson, Saint Mary's University, E-mail. The variables are pclass, age, sex, survived. These data frames are useful for demonstrating many of the functions in Hmisc as well as demonstrating binary logistic regression analysis using the Design library. For more details and references see Simonoff, Jeffrey S (1997): The "unusual episode" and a second statistics course. J Statistics Education, Vol. 5 No. 1.

Downloaded from openml.org.

In [3]:
# Leemos los datos desde un fichero local

# Deben poner la ruta de la base de datos con doble \\
# OJO instalar en el CMD pip install pyarrow para que lea el archivo parquet

titanic = pd.read_parquet('C:\\Users\\jlpad\\Desktop\\BBDD\\titanic.parquet', engine = 'pyarrow')

In [4]:
# Mostramos las 10 primeras líneas del DataFrame

titanic.head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
5,1.0,1,"Anderson, Mr. Harry",male,48.0000,0.0,0.0,19952,26.5500,E12,S,3,NaN,"New York, NY"
6,1.0,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1.0,0.0,13502,77.9583,D7,S,10,NaN,"Hudson, NY"
7,1.0,0,"Andrews, Mr. Thomas Jr",male,39.0000,0.0,0.0,112050,0.0000,A36,S,None,NaN,"Belfast, NI"
8,1.0,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2.0,0.0,11769,51.4792,C101,S,D,NaN,"Bayside, Queens, NY"
9,1.0,0,"Artagaveytia, Mr. Ramon",male,71.0000,0.0,0.0,PC 17609,49.5042,None,C,None,22.0,"Montevideo, Uruguay"


In [5]:
# .info() nos proporciona información sobre el tipo de datos y el tamaño de cada una de las columnas
# Como puede verse, los tipos numéricos se han interpretado como tales, los valores que solo tienen
# un número pequeño de opciones están como tipo categórico, y los que tienen muchas opciones pero no
# son numéricos están como "object", de manera que no hay que modificar nada. También es interesante
# ver qué campos tineen valores nulos o no (se muestra el número de valores no nulos por cada columna,
# si es igual al total de valores, que se menciona al principio del resultado, significa que no hay
# valores nulos. También podría utilizarse el método .count() del dataframe para lo mismo)

titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   pclass     1309 non-null   float64 
 1   survived   1309 non-null   category
 2   name       1309 non-null   object  
 3   sex        1309 non-null   category
 4   age        1046 non-null   float64 
 5   sibsp      1309 non-null   float64 
 6   parch      1309 non-null   float64 
 7   ticket     1309 non-null   object  
 8   fare       1308 non-null   float64 
 9   cabin      295 non-null    object  
 10  embarked   1307 non-null   category
 11  boat       486 non-null    object  
 12  body       121 non-null    float64 
 13  home.dest  745 non-null    object  
dtypes: category(3), float64(6), object(5)
memory usage: 116.8+ KB


In [6]:
# Con .describe() vemos un resumen de los datos. La información mostrada es diferente en función de que las columnas sean numéricas (que es la única información que se muestra por defecto) o que sean de otros
# tipos (que es posible incluir mediante .describe(include = 'all')  Sin embargo, generalmente es más sencillo ver cada tipo de columnas por separado)

# Descripción de las columnas numéricas...

titanic.describe()

,pclass,age,sibsp,parch,fare,body
count,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,29.881135,0.498854,0.385027,33.295479,160.809917
std,0.837836,14.413500,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,2.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,80.000000,8.000000,9.000000,512.329200,328.000000


In [11]:
# Descripción de las columnas de tipo categórico.

# 'count' se refiere al número total de casos no nulos, 
# 'unique' al número de opciones posibles,
# 'top' a la opción más frecuente 
# 'freq' a la frecuencia de la opción más frecuente

titanic.describe(include = 'category')

,survived,sex,embarked
count,1309,1309,1307
unique,2,2,3
top,0,male,S
freq,809,843,914


In [12]:
# Descripción de las columnas tipo object

titanic.describe(include = 'object')

,name,ticket,cabin,boat,home.dest
count,1309,1309,295,486,745
unique,1307,929,186,27,369
top,"Connolly, Miss. Kate",CA. 2343,C23 C25 C27,13,"New York, NY"
freq,2,11,6,39,64


## Análisis y tratamiento de (algunos) problemas de datos 

A continuación vamos a tratar de hacer algunos análisis sencillos pero en los que nos encontramos algunos problemas de calidad de datos, para ver cómo abordarlos.

Observando los resultado de la descripción de los datos en columnas de tipo "object", vemos que hay 1309 valores no nulos (o sea, que los datos están completos, ya que hay 1309 casos), pero solo 1307 valores únicos: esto quiere decir que hay valores repetidos. ¿Estarán repetidos los datos? ¿Podemos usar el nombre del pasajero como identificador único?

In [13]:
# Mostramos las columnas del DataFrame que tienen el campo "name" duplicado (el añadir keep = False es para que se muestren todos los valores: por defecto, .duplicated() marca como "no duplicado" 
# al primer caso que se encuentra y como "duplicado" a cualquiera que sea igual, de manera que si se seleccionan todos los registros que no sean "duplicados" se tiene uno de cada, no solo los que no están duplicados)

titanic[titanic.name.duplicated(keep = False)]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
725,3.0,1,"Connolly, Miss. Kate",female,22.0,0.0,0.0,370373,7.7500,None,Q,13,NaN,Ireland
726,3.0,0,"Connolly, Miss. Kate",female,30.0,0.0,0.0,330972,7.6292,None,Q,None,NaN,Ireland
924,3.0,0,"Kelly, Mr. James",male,34.5,0.0,0.0,330911,7.8292,None,Q,None,70.0,None
925,3.0,0,"Kelly, Mr. James",male,44.0,0.0,0.0,363592,8.0500,None,S,None,NaN,None


Vemos que existen dos nombres duplicados, pero corresponden a personas distintas (una de las Kate Connolly sobrevivió al naufragio y otra no!), así que no se deben eliminar. No se debe utilizar el nombre como identificador único en este caso (y en general, casi nunca) ya que podría dar problema. Si no fueran personas distintas, sino un problema de duplicación de datos, se podría hacer simplemente

    titanic.drop_duplicates(['name'])
    
para eleminiar aquellos casos que tengan el mismo nombre, aunque suele ser recomendable hacer algo como

    titanic.drop_duplicates()
    
para eleminar solamnete aquellos registros que tengan todos los datos iguales (si es que es eso lo que se desea, claro)

Como vemos aquí, la eliminación de duplicados, como casi todas las tareas de limpieza de datos, requiere un análisis minucioso de las acciones a tomar, para no empeorar los datos en lugar de mejorarlos. En este caso se trata de un conjunto de datos ya estandarizado y muy trabajado, de manera que no es esperable que haya errores fácilmente subsanables como duplicidades. Sin embargo, el tratamiento de problemas de calidad de datos no solo está orientado a "corregir" errores los datos, sino a mejorar alguna característica de calidad que los haga más utilizables.

Como siguiente ejemplo, vamos a tratar de analizar la compra de tickets. En la descripción de los datos tipo object vemos que el campo "ticket" está presente para todos los 1309 casos, pero hay solo hay  929 valores distintos, lo que quiere decir que algunos tickets incluían a varios pasajeros.

In [14]:
# Mostramos los datos de los casos con el campo 'ticket' que esten duplicados
# podemos ver que habia un solo ticket por familia o grupo de personas que viajaban juntas, 
# en este caso habian 596 pasajeros con tickets conjuntos
ticket_multiple = titanic.ticket.duplicated(keep = False)
titanic[ticket_multiple]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,3.0,0,"Yasbeck, Mr. Antoni",male,27.0000,1.0,0.0,2659,14.4542,None,C,C,NaN,None
1300,3.0,1,"Yasbeck, Mrs. Antoni (Selini Alexander)",female,15.0000,1.0,0.0,2659,14.4542,None,C,None,NaN,None
1303,3.0,0,"Yousseff, Mr. Gerious",male,NaN,0.0,0.0,2627,14.4583,None,C,None,NaN,None
1304,3.0,0,"Zabour, Miss. Hileni",female,14.5000,1.0,0.0,2665,14.4542,None,C,None,328.0,None


In [15]:
# Para los casos con el campo "ticket" duplicado, seleccionamos el campo ticket y mostramos cuantos valores únicos hay

titanic[ticket_multiple].ticket.nunique()

216

Vemos que hay 596 pasajeros con tickets conjuntos, y 216 de esos tickets. Una simple inspección de los pocos datos mostramos nos permite intuir que toda una familia podía estar incluida en un ticket (según la costumbre de la época, la esposa tomaba el apellido del marido, aunque en algún caso podría ser que se tratara de hermanos, o de una simple coincidencia, como los nombres duplicados).

Queremos analizar esos tickets conjutos: cuantos miembros de una misma familia había, etc. Lamentablemente, no disponemos del nombre y el apellido por separado, aunque los nombres completos estén: se trata de un problema de **validez** de datos, ya que no están en el formato que nos gustaría tenerlos. Podemos tratar de resolver este problema ("problema" para nosotros, que estamos interesados en este análisis concreto; no necesariamente es "un problema" de los datos, no es que los datos estén mal, es que no están en el formato adecuado para nuestros fines)

In [16]:
# creamos dos nuevos campos en nuestro dataframe 'Apellido' y'Nombre' para analizar cuantos integrantes de una misma familia habian
# como vemos el campo 'name' se divide el nombre del apellido por una 'COMA' asi que lo dividimos en dos nuevos campos con la funcion 'SPLIT'

titanic[['apellido', 'nombre']] = titanic.name.str.split(',', expand = True)
titanic.head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,apellido,nombre
0,1.0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",Allen,Miss. Elisabeth Walton
1,1.0,1,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",Allison,Master. Hudson Trevor
2,1.0,0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON",Allison,Miss. Helen Loraine
3,1.0,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON",Allison,Mr. Hudson Joshua Creighton
4,1.0,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON",Allison,Mrs. Hudson J C (Bessie Waldo Daniels)
5,1.0,1,"Anderson, Mr. Harry",male,48.0000,0.0,0.0,19952,26.5500,E12,S,3,NaN,"New York, NY",Anderson,Mr. Harry
6,1.0,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1.0,0.0,13502,77.9583,D7,S,10,NaN,"Hudson, NY",Andrews,Miss. Kornelia Theodosia
7,1.0,0,"Andrews, Mr. Thomas Jr",male,39.0000,0.0,0.0,112050,0.0000,A36,S,None,NaN,"Belfast, NI",Andrews,Mr. Thomas Jr
8,1.0,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2.0,0.0,11769,51.4792,C101,S,D,NaN,"Bayside, Queens, NY",Appleton,Mrs. Edward Dale (Charlotte Lamson)
9,1.0,0,"Artagaveytia, Mr. Ramon",male,71.0000,0.0,0.0,PC 17609,49.5042,None,C,None,22.0,"Montevideo, Uruguay",Artagaveytia,Mr. Ramon


Ahora ya es posible hacer esos análisis...

In [17]:
# Filtramos los datos correspondientes a tickets múltiples, agrupamos por ticket y apellido, calculamos 
#una serie de agregados de algunas columnas a las que poner nombres, y asignamos ese resultado a un
# nuevo DataFrame llamado "grupos"

grupos = titanic[ticket_multiple].groupby(['ticket', 'apellido'])\
                                 .agg(nombres = pd.NamedAgg('nombre', lambda x: x.tolist()),
                                      
                                      masculino = pd.NamedAgg('sex', lambda x: (x == 'male').sum()),
                                      femenino = pd.NamedAgg('sex', lambda x: (x == 'female').sum()),
                                      
                                      edades = pd.NamedAgg('age', lambda x: x.tolist()),
                                      
                                      mayores_de_edad = pd.NamedAgg('age', lambda x: (x >= 16).sum()),
                                      menores_de_edad = pd.NamedAgg('age', lambda x: (x < 16).sum()),
                                      
                                      personas_con_apellido = pd.NamedAgg('apellido', lambda x: x.count()))\
                                 .reset_index()

# Creamos dos nuevas columnas con información relevante: el número de apellidos distintos en cada grupo, y
# el número de personas en cada grupo (como hicimos la agrupación poer ticket y apellido al mismo tiempo, estos
# valores no se podían calcular de manera sencilla en la misma sentencia; también habría sido posible hacer los
# mismo calculos haciendo primero un groupby por ticket, calculando algunos valores, y luego un groupby por apellido)
grupos['apellidos_en_grupo'] = grupos.groupby('ticket').apellido.transform(lambda x: x.count())
grupos['personas_en_grupo'] = grupos.groupby('ticket').personas_con_apellido.transform(lambda x: x.sum())
grupos

,ticket,apellido,nombres,masculino,femenino,edades,mayores_de_edad,menores_de_edad,personas_con_apellido,apellidos_en_grupo,personas_en_grupo
0,110152,Cherry,[ Miss. Gladys],0,1,[30.0],1.0,0.0,1,3,3
1,110152,Maioni,[ Miss. Roberta],0,1,[16.0],1.0,0.0,1,3,3
2,110152,Rothes,[ the Countess. of (Lucy Noel Martha Dyer-Edwa...,0,1,[33.0],1.0,0.0,1,3,3
3,110413,Taussig,"[ Miss. Ruth, Mr. Emil, Mrs. Emil (Tillie Ma...",1,2,"[18.0, 52.0, 39.0]",3.0,0.0,3,1,3
4,110465,Clifford,[ Mr. George Quincy],1,0,[nan],0.0,0.0,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...
306,STON/O2. 3101279,Hakkarainen,"[ Mr. Pekka Pietari, Mrs. Pekka Pietari (Elin...",1,1,"[28.0, 24.0]",2.0,0.0,2,1,2
307,W./C. 6607,Johnston,"[ Master. William Arthur 'Willie', Miss. Cath...",2,2,"[nan, nan, nan, nan]",0.0,0.0,4,1,4
308,W./C. 6608,Ford,"[ Miss. Doolina Margaret 'Daisy', Miss. Robin...",2,3,"[21.0, 9.0, 18.0, 16.0, 48.0]",4.0,1.0,5,1,5
309,W.E.P. 5734,Chaffee,"[ Mr. Herbert Fuller, Mrs. Herbert Fuller (Ca...",1,1,"[46.0, 47.0]",2.0,0.0,2,1,2


### Nota técnica de programación: funciones lambda

En el código anterior se utilizan funciones lambda para calcular los agragegados y las transformaciones. Una "lambda" no es más que una pequeña función anónima. Generalmente se usan cuando una función solo se va a usar una sola vez, de manera que resulta más breve y más claro no definirla por separado. Por ejemplo, los siguientes dos ejemplos de código (que calcula, para cada clase, la edad por debajo de la cual está el 90 porciento de los pasajeros de esa clase), son (casi) equivalentes:

    # Ejemplo 1: usando una "lambda"
    grupos.groupby('pclass').age.agg(lambda x: x.quantile(0.90)

    # Ejemplo 2: definiendo una función
    def percentil90(serie):
        return serie.quantile(0.90)
    grupos.groupby('pclass').age.agg(percentile90)

La diferencia principal entre los dos ejemplos, aparte de la longuitud, es que después de ejecutar el segundo queda definida la función percentil90 de ahí en adelante (de manera general, es preferible no dejar variables y funciones definidas que no se vayan a utilizar más, tanto por rendimiento como para evitar errores)

En el código del ejemplo, la función (ya sea definida mediante una lambda o externamente) se pasa como parámetro a otra función. Esto es una práctica bastante común, y la librería Pandas hace uso frecuente de la misma. Por ejemplo, los métodos .agg() o .transform() utilizados anteriormente, toman cada grupo seleccionado mediante .groupby() y le aplican la función que se les proporciona (la diferencia entre los mismos es que .agg() genera un agregado, por ejemplo la suma o la media de todos los valores de cada grupo, mientras que .transform() transforma los datos pero los deja con la misma dimensión, por ejemplo normalizandolos o calculando algún valor para adjuntar como columna adicional para facilitar cálculos posteriores)

Para más detalles sobre el uso de funciones lambda, consultar la documentación estándar de python o cualquier manual de programación en python


In [18]:
# Número de apellidos ditintos en función del tamaño del grupo que viaja con el mismo billete.
# Como se puede apreciar, el caso mas frecuente es el de un grupo de dos personas con el mismo apellido
# (seguramente, un matrimonio, o un progenitor con su descendiente), pero hay bastantes casos diferentes

#'CROSSTAB' de pandas nos hace una tabla cruzada de 2 valores (columnas)  

pd.crosstab(grupos.apellidos_en_grupo, grupos.personas_en_grupo)

personas_en_grupo,2,3,4,5,6,7,8,11
apellidos_en_grupo,,,,,,,,
1,99,33,7,3,3,2,1,1
2,66,22,4,2,0,2,0,0
3,0,15,15,6,3,3,0,0
4,0,0,8,4,0,0,0,0
5,0,0,0,0,0,5,0,0
7,0,0,0,0,0,0,7,0


In [19]:
# Para los grupos en los que todos los miembros tengan el mismo apellido, calculamos
# cuantas personas hay de cada sexo. Se ve que lo más frecuente es que viajen parejas
# de hombre y mujer (80 casos)

grupos_1_apellido = grupos[grupos.apellidos_en_grupo == 1]
pd.crosstab(grupos_1_apellido.femenino, grupos_1_apellido.masculino)

masculino,0,1,2,3,5,6
femenino,,,,,,
0,0,0,5,3,0,0
1,0,80,10,0,1,0
2,14,18,3,0,1,0
3,2,3,2,1,1,0
4,1,1,1,0,0,0
5,0,0,1,0,0,1


In [20]:
# Hacemos lo mismo, pero esta vez con el numero de mayores y menores de 16 años...
pd.crosstab(grupos_1_apellido.menores_de_edad, grupos_1_apellido.mayores_de_edad)

mayores_de_edad,0.0,1.0,2.0,3.0,4.0,6.0
menores_de_edad,,,,,,
0.0,14,5,72,4,0,1
1.0,2,13,14,0,1,0
2.0,2,9,4,0,0,0
3.0,0,2,0,0,0,0
4.0,0,1,1,0,0,0
5.0,0,1,2,1,0,0


¡Hemos encontrado un error! Los resultados no tienen sentido: aparecen 14 casos, por ejemplo, en lo que un ticket para dos o más personas no incluye ninguna persona de más de 16 años y ninguna persona de menos de 16 años, lo que no tiene sentido. Si volvemos a la información inicial sobre los datos, vemos cual es el problema: el campo "age" del DataFrame original solo tiene valores válidos para 1046 de los 1309 casos, el resto son valores nulos. Este es un error típico causado por no tener en cuenta un problema de calidad de datos.

Podemos comprobar fácilmente el error...

In [21]:
# Contamos para cuantos de las filas del DataFrame "grupos" coincide en número de personas (que siempre está bien)
# con la suma de menores de edad y mayores de edad (que solo incluye a las personas para las que se tiene la edad)
(grupos.mayores_de_edad + grupos.menores_de_edad == grupos.personas_en_grupo).value_counts()

False    185
True     126
dtype: int64

¿Cómo podría solucionarse este problema? Hay distintas posibilidades, según nuestro objetivo. Podríamos rellenar los datos con una edad estimada, por ejemplo, con la media de todas las edades, pero en este caso concreto no sería una buena opción, porque haría que todas esas personas pasaran a aparecer como "mayores de edad" y descompensaría los datos de una manera poco realista. Una opción mejor, en este caso, es no considerar los casos en los que al menos para una de las personas del grupo no se tenga la edad: de esta manera veremos solo la tabla para aquellos grupos para los que tenemos información completa, que no nos da unos valores totales verdaderos pero sí nos permite hacernos una idea de la proporción entre las alternativas.

In [22]:
# Filtramos los datos para quedarnos únicamente con aquellos para los que tengamos información completa del grupo

g_1_a_filtrado = grupos_1_apellido[(grupos_1_apellido.mayores_de_edad + grupos_1_apellido.menores_de_edad) == grupos_1_apellido.personas_con_apellido]

pd.crosstab(g_1_a_filtrado.menores_de_edad, g_1_a_filtrado.mayores_de_edad)

mayores_de_edad,0.0,1.0,2.0,3.0,4.0,6.0
menores_de_edad,,,,,,
0.0,0,0,72,4,0,1
1.0,0,12,14,0,1,0
2.0,1,9,4,0,0,0
3.0,0,2,0,0,0,0
4.0,0,1,1,0,0,0
5.0,0,1,2,1,0,0


Los resultados, naturalmente, no son perfectos (no tenemos información suficiente para que lo sean), pero nos dan una información útil, si sabemos interpretarla: el caso más común es el de una pareja de adultos solos, seguido de uno o dos adultos acompañados de un niño, o de dos o más en algunos casos. Solo hay un caso de dos menores viajando solos, y grupos de 3 o más adultos vaijando con un menor también son muy infrecuentes.

## Detección de anomalías 

Vamos ahora a hacer un análisis sencillo de cómo se relaciona la clase (primera, segunda o tercera) con el precio del billete

In [23]:
for clase in [1, 2, 3]:
    print(f'--- CLASE {clase} ---')
    print(titanic[titanic.pclass == clase].fare.describe())
    print(clase)

--- CLASE 1 ---
count    323.000000
mean      87.508992
std       80.447178
min        0.000000
25%       30.695800
50%       60.000000
75%      107.662500
max      512.329200
Name: fare, dtype: float64
1
--- CLASE 2 ---
count    277.000000
mean      21.179196
std       13.607122
min        0.000000
25%       13.000000
50%       15.045800
75%       26.000000
max       73.500000
Name: fare, dtype: float64
2
--- CLASE 3 ---
count    708.000000
mean      13.302889
std       11.494358
min        0.000000
25%        7.750000
50%        8.050000
75%       15.245800
max       69.550000
Name: fare, dtype: float64
3


Vemos que el precio mínimo pagado, en los tres casos, es cero: parece que hubo pasajeros que viajaban invitados, en todas las clases. Vamos a ver cuántos eran, y a eliminarlos de la muestra para tener una visión más realista de los precios reales de los tickets

In [24]:
titanic[titanic.fare == 0].groupby('pclass').ticket.count()

pclass
1.0    7
2.0    6
3.0    4
Name: ticket, dtype: int64

In [25]:
for clase in [1, 2, 3]:
    print(f'--- CLASE {clase} ---')
    print(titanic[(titanic.fare != 0) &(titanic.pclass == clase)].fare.describe())
    print(clase)

--- CLASE 1 ---
count    316.000000
mean      89.447482
std       80.259713
min        5.000000
25%       31.682275
50%       61.379200
75%      108.900000
max      512.329200
Name: fare, dtype: float64
1
--- CLASE 2 ---
count    271.000000
mean      21.648108
std       13.382064
min        9.687500
25%       13.000000
50%       15.050000
75%       26.000000
max       73.500000
Name: fare, dtype: float64
2
--- CLASE 3 ---
count    704.000000
mean      13.378473
std       11.483004
min        3.170800
25%        7.750000
50%        8.050000
75%       15.245800
max       69.550000
Name: fare, dtype: float64
3


Hay una variabilidad muy grande en los precios de los tickets. Vamos a ver los casos más extremos...

In [26]:
# Para ver los pasajeros de primera clase que menos pagaron, seleccionamos esos pasajeros, 
# ordenamos por precio en sentido descendiente, y vemos los 10 primero casos que aparecen

titanic[(titanic.fare != 0) & (titanic.pclass == 1)].sort_values(by = 'fare').head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,apellido,nombre
51,1.0,0,"Carlsson, Mr. Frans Olof",male,33.0,0.0,0.0,695,5.0000,B51 B53 B55,S,None,NaN,"New York, NY",Carlsson,Mr. Frans Olof
75,1.0,0,"Colley, Mr. Edward Pomeroy",male,47.0,0.0,0.0,5727,25.5875,E58,S,None,NaN,"Victoria, BC",Colley,Mr. Edward Pomeroy
79,1.0,1,"Cornell, Mrs. Robert Clifford (Malvina Helen L...",female,55.0,2.0,0.0,11770,25.7000,C101,S,2,NaN,"New York, NY",Cornell,Mrs. Robert Clifford (Malvina Helen Lamson)
219,1.0,1,"Omont, Mr. Alfred Fernand",male,NaN,0.0,0.0,F.C. 12998,25.7417,None,C,7,NaN,"Paris, France",Omont,Mr. Alfred Fernand
15,1.0,0,"Baumann, Mr. John D",male,NaN,0.0,0.0,PC 17318,25.9250,None,S,None,NaN,"New York, NY",Baumann,Mr. John D
288,1.0,1,"Swift, Mrs. Frederick Joel (Margaret Welles Ba...",female,48.0,0.0,0.0,17466,25.9292,D17,S,8,NaN,"Brooklyn, NY",Swift,Mrs. Frederick Joel (Margaret Welles Barron)
181,1.0,1,"Leader, Dr. Alice (Farnham)",female,49.0,0.0,0.0,17465,25.9292,D17,S,8,NaN,"New York, NY",Leader,Dr. Alice (Farnham)
171,1.0,0,"Jones, Mr. Charles Cresson",male,46.0,0.0,0.0,694,26.0000,None,S,None,80.0,"Bennington, VT",Jones,Mr. Charles Cresson
172,1.0,0,"Julian, Mr. Henry Forbes",male,50.0,0.0,0.0,113044,26.0000,E60,S,None,NaN,London,Julian,Mr. Henry Forbes
217,1.0,0,"Nicholson, Mr. Arthur Ernest",male,64.0,0.0,0.0,693,26.0000,None,S,None,263.0,"Isle of Wight, England",Nicholson,Mr. Arthur Ernest


In [27]:
# Seleccionamos los pasajeros de clase 3, ordenamos por precio en sentido descendiente, y vemos los 10 primero casos que aparecen

titanic[titanic.pclass == 3].sort_values(by = 'fare', ascending = False).head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,apellido,nombre
1171,3.0,0,"Sage, Master. William Henry",male,14.5,8.0,2.0,CA. 2343,69.55,None,S,None,67.0,None,Sage,Master. William Henry
1175,3.0,0,"Sage, Miss. Stella Anna",female,NaN,8.0,2.0,CA. 2343,69.55,None,S,None,NaN,None,Sage,Miss. Stella Anna
1172,3.0,0,"Sage, Miss. Ada",female,NaN,8.0,2.0,CA. 2343,69.55,None,S,None,NaN,None,Sage,Miss. Ada
1179,3.0,0,"Sage, Mr. John George",male,NaN,1.0,9.0,CA. 2343,69.55,None,S,None,NaN,None,Sage,Mr. John George
1178,3.0,0,"Sage, Mr. George John Jr",male,NaN,8.0,2.0,CA. 2343,69.55,None,S,None,NaN,None,Sage,Mr. George John Jr
1177,3.0,0,"Sage, Mr. Frederick",male,NaN,8.0,2.0,CA. 2343,69.55,None,S,None,NaN,None,Sage,Mr. Frederick
1176,3.0,0,"Sage, Mr. Douglas Bullen",male,NaN,8.0,2.0,CA. 2343,69.55,None,S,None,NaN,None,Sage,Mr. Douglas Bullen
1180,3.0,0,"Sage, Mrs. John (Annie Bullen)",female,NaN,1.0,9.0,CA. 2343,69.55,None,S,None,NaN,None,Sage,Mrs. John (Annie Bullen)
1174,3.0,0,"Sage, Miss. Dorothy Edith 'Dolly'",female,NaN,8.0,2.0,CA. 2343,69.55,None,S,None,NaN,None,Sage,Miss. Dorothy Edith 'Dolly'
1170,3.0,0,"Sage, Master. Thomas Henry",male,NaN,8.0,2.0,CA. 2343,69.55,None,S,None,NaN,None,Sage,Master. Thomas Henry


Parece que hay muchos pasajeros que pagaron el precio de 69.55 en tercera clase, pero si observamos vemos que todos ellos están en el mismo ticket. Esto nos hace pensar que el precio que aparece en los datos no es el precio por pasajero, sino el precio por el ticket completo. Aunque la documentación asociada a los datos no lo aclara, algunas pruebas parecen confirmar esa hipótesis

In [28]:
print('\:egg:')

\:egg:


In [29]:
# Para cada grupo de pasajeros con el mismo ticket, comprobamos la diferencia entre el precio máximo que aparece
# para alguno de ellos y el precio mínimo, y mostramos los valores.
# Vemos que en todos los casos esa diferencia es cero, lo que quiere decir que todo los pasajeros con el mismo ticket
# tienen el mismo precio... lo que sugiere que ese es el precio del grupo, no el precio por persona

(titanic.groupby('ticket').fare.max() - titanic.groupby('ticket').fare.max()).value_counts()

0.0    928
Name: fare, dtype: int64

In [30]:
# Vamos a crear una nueva variable, "precio_imputado", que reparte el precio del ticket entre las distintas personas

titanic['precio_imputado'] = titanic.groupby('ticket').fare.transform(lambda x: x / x.count())

In [31]:
for clase in [1, 2, 3]:
    print(f'--- CLASE {clase} ---')
    print(titanic[(titanic.precio_imputado != 0) &(titanic.pclass == clase)].precio_imputado.describe())
    print(clase)

--- CLASE 1 ---
count    316.000000
mean      34.661682
std       14.675124
min        5.000000
25%       26.550000
50%       30.000000
75%       39.133350
max      128.082300
Name: precio_imputado, dtype: float64
1
--- CLASE 2 ---
count    271.000000
mean      11.663652
std        2.031927
min        5.250000
25%       10.500000
50%       12.650000
75%       13.000000
max       16.000000
Name: precio_imputado, dtype: float64
2
--- CLASE 3 ---
count    704.000000
mean       7.370788
std        1.367423
min        3.170800
25%        7.061975
50%        7.750000
75%        7.925000
max       19.966700
Name: precio_imputado, dtype: float64
3


Parece que resultan unos valores más equilibrados (y muy concentrados en la zona central), así que vamos a suponer que la hipótesis es verdadera. Aún así, parece que hay algunos valores muy altos tercera clase y algunos valores muy bajos en primera. 

In [32]:
#Veamos los pasajeros que más pagaron por un ticket de tercera clase...

titanic[titanic.pclass == 3].sort_values(by = 'precio_imputado', ascending = False).head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,apellido,nombre,precio_imputado
842,3.0,0,"Hagland, Mr. Ingvald Olai Olsen",male,NaN,1.0,0.0,65303,19.9667,None,S,None,NaN,None,Hagland,Mr. Ingvald Olai Olsen,19.9667
843,3.0,0,"Hagland, Mr. Konrad Mathias Reiersen",male,NaN,1.0,0.0,65304,19.9667,None,S,None,NaN,None,Hagland,Mr. Konrad Mathias Reiersen,19.9667
743,3.0,0,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0.0,0.0,7552,10.5167,None,S,None,NaN,"Norrlot, Sweden Chicago, IL",Dahlberg,Miss. Gerda Ulrika,10.5167
744,3.0,0,"Dakic, Mr. Branko",male,19.0,0.0,0.0,349228,10.1708,None,S,None,NaN,Austria,Dakic,Mr. Branko,10.1708
1260,3.0,1,"Turja, Miss. Anna Sofia",female,18.0,0.0,0.0,4138,9.8417,None,S,15,NaN,None,Turja,Miss. Anna Sofia,9.8417
1227,3.0,0,"Strandberg, Miss. Ida Sofia",female,22.0,0.0,0.0,7553,9.8375,None,S,None,NaN,None,Strandberg,Miss. Ida Sofia,9.8375
907,3.0,0,"Jussila, Miss. Katriina",female,20.0,1.0,0.0,4136,9.8250,None,S,None,NaN,None,Jussila,Miss. Katriina,9.8250
908,3.0,0,"Jussila, Miss. Mari Aina",female,21.0,1.0,0.0,4137,9.8250,None,S,None,NaN,None,Jussila,Miss. Mari Aina,9.8250
1261,3.0,1,"Turkula, Mrs. (Hedwig)",female,63.0,0.0,0.0,4134,9.5875,None,S,15,NaN,None,Turkula,Mrs. (Hedwig),9.5875
943,3.0,0,"Laitinen, Miss. Kristina Sofia",female,37.0,0.0,0.0,4135,9.5875,None,S,None,NaN,None,Laitinen,Miss. Kristina Sofia,9.5875


Vemos que hay dos pasajeros, que además comparten apellido, que pagaron 19.96 cada uno por su ticket, cuando los siguientes billetes de tercera más caros costaron exáctamente la mitad. Esto nos hace sospechar que se trate de un error: tal vez el número de ticket es incorrecto, debería ser el mismo, y el precio imputado a cada uno sería la mitad. El hecho de el precio esté fuera de rango y sea improbable (es el doble del siguiente precio pagado, es mayor que el precio pagado por ningún pasajero de segunda, e incluso algunos de primera viajaron por menos de eso) hace que sea una **anomalía**. Si es un error o no, eso ya depende del contenido de los datos y necesita ser determinado por otros medios.

(En realidad, en este caso, es posible consultar los datos existentes sobre las víctimas del Titanic. Tanto Ingvald Hagland como Konrad Hagland, hermanos, tenían previsto haber viajado con sendos acompañantes, que finalmente no embarcaron y por eso no aparecen en la lista de pasajeros. El precio por persona al que compraron los billetes fue, por lo tanto, la mitad del que aparece en nuestra tabla) 

¿Se debería corregir ese dato? Como siempre, depende del uso que se le quiera dar. Por ejemplo: si se quiere hacer un análisis de cuales eran los precios de los billetes, lo más correcto sería corregirlo. Si se quiere sumar lo pagado por cada viajero para calcular el total de ingresos de la compañía naviera, lo correcto sería no corregirlo. La anómalia es una característica del dato, pero la corrección o no, o el descubrimiento de un hecho interesante, es dependiente de la realidad y del modelo.

In [33]:
titanic[titanic.pclass == 1].sort_values(by = 'precio_imputado', ascending = False).head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,apellido,nombre,precio_imputado
50,1.0,1,"Cardeza, Mrs. James Warburton Martinez (Charlo...",female,58.0,0.0,1.0,PC 17755,512.3292,B51 B53 B55,C,3,NaN,"Germantown, Philadelphia, PA",Cardeza,Mrs. James Warburton Martinez (Charlotte Ward...,128.082300
302,1.0,1,"Ward, Miss. Anna",female,35.0,0.0,0.0,PC 17755,512.3292,None,C,3,NaN,None,Ward,Miss. Anna,128.082300
183,1.0,1,"Lesurer, Mr. Gustave J",male,35.0,0.0,0.0,PC 17755,512.3292,B101,C,3,NaN,None,Lesurer,Mr. Gustave J,128.082300
49,1.0,1,"Cardeza, Mr. Thomas Drake Martinez",male,36.0,0.0,1.0,PC 17755,512.3292,B51 B53 B55,C,3,NaN,"Austria-Hungary / Germantown, Philadelphia, PA",Cardeza,Mr. Thomas Drake Martinez,128.082300
17,1.0,1,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",female,50.0,0.0,1.0,PC 17558,247.5208,B58 B60,C,6,NaN,"Montreal, PQ",Baxter,Mrs. James (Helene DeLaudeniere Chaput),82.506933
16,1.0,0,"Baxter, Mr. Quigg Edmond",male,24.0,0.0,1.0,PC 17558,247.5208,B58 B60,C,None,NaN,"Montreal, PQ",Baxter,Mr. Quigg Edmond,82.506933
97,1.0,1,"Douglas, Mrs. Frederick Charles (Mary Helene B...",female,27.0,1.0,1.0,PC 17558,247.5208,B58 B60,C,6,NaN,"Montreal, PQ",Douglas,Mrs. Frederick Charles (Mary Helene Baxter),82.506933
72,1.0,1,"Clark, Mrs. Walter Miller (Virginia McDowell)",female,26.0,1.0,0.0,13508,136.7792,C89,C,4,NaN,"Los Angeles, CA",Clark,Mrs. Walter Miller (Virginia McDowell),68.389600
71,1.0,0,"Clark, Mr. Walter Miller",male,27.0,1.0,0.0,13508,136.7792,C89,C,None,NaN,"Los Angeles, CA",Clark,Mr. Walter Miller,68.389600
119,1.0,1,"Frauenthal, Dr. Henry William",male,50.0,2.0,0.0,PC 17611,133.6500,None,S,5,NaN,"New York, NY",Frauenthal,Dr. Henry William,66.825000


Los precios más altos pagados en primera también corresponden a tickets con varias personas. En este caso puede verse además, observando el campo "cabin", que se trata de reservas de varios camarotes, lo que puede justificar el precio, y probablemente no debería considerarse erróneo (nuevamente, consultando los datos de pasajeros se puede leer de Mrs. Charlotte Cardeza, una rica heredera de constumbre extravagantes que viajaba con su hijo y sus criados, y transportaba una gran cantidad de equipaje)

In [34]:
titanic[(titanic.fare != 0) & (titanic.pclass == 1)].sort_values(by = 'precio_imputado').head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,apellido,nombre,precio_imputado
51,1.0,0,"Carlsson, Mr. Frans Olof",male,33.0,0.0,0.0,695,5.0,B51 B53 B55,S,None,NaN,"New York, NY",Carlsson,Mr. Frans Olof,5.000
46,1.0,0,"Cairns, Mr. Alexander",male,NaN,0.0,0.0,113798,31.0,None,S,None,NaN,None,Cairns,Mr. Alexander,15.500
258,1.0,1,"Serepeca, Miss. Augusta",female,30.0,0.0,0.0,113798,31.0,None,C,4,NaN,None,Serepeca,Miss. Augusta,15.500
188,1.0,1,"Lines, Mrs. Ernest H (Elizabeth Lindsey James)",female,51.0,0.0,1.0,PC 17592,39.4,D28,S,9,NaN,"Paris, France",Lines,Mrs. Ernest H (Elizabeth Lindsey James),19.700
187,1.0,1,"Lines, Miss. Mary Conover",female,16.0,0.0,1.0,PC 17592,39.4,D28,S,9,NaN,"Paris, France",Lines,Miss. Mary Conover,19.700
147,1.0,0,"Harrington, Mr. Charles H",male,NaN,0.0,0.0,113796,42.4,None,S,None,NaN,None,Harrington,Mr. Charles H,21.200
211,1.0,0,"Moore, Mr. Clarence Bloomfield",male,47.0,0.0,0.0,113796,42.4,None,S,None,NaN,"Washington, DC",Moore,Mr. Clarence Bloomfield,21.200
154,1.0,0,"Hays, Mr. Charles Melville",male,55.0,1.0,1.0,12749,93.5,B69,S,None,307.0,"Montreal, PQ",Hays,Mr. Charles Melville,23.375
225,1.0,0,"Payne, Mr. Vivian Ponsonby",male,23.0,0.0,0.0,12749,93.5,B24,S,None,NaN,"Montreal, PQ",Payne,Mr. Vivian Ponsonby,23.375
155,1.0,1,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",female,52.0,1.0,1.0,12749,93.5,B69,S,3,NaN,"Montreal, PQ",Hays,Mrs. Charles Melville (Clara Jennings Gregg),23.375


Finalmente, el precio más bajo pagado en primera también es significativamente más bajo que los siguientes, y puede considerarse un dato a corregir o no según la utilización que quiera hacerse de los datos (en este caso se trata de un marino de alta graduación que no pudo regresar en su propio barco a los EEUU y tuvo que viajar en el Titanic, por lo que probablemente el precio muy reducido sea consecuencia de algún tipo de acuerdo o cortesía entre las empresas navieras)

In [35]:
grupos.head()

,ticket,apellido,nombres,masculino,femenino,edades,mayores_de_edad,menores_de_edad,personas_con_apellido,apellidos_en_grupo,personas_en_grupo
0,110152,Cherry,[ Miss. Gladys],0,1,[30.0],1.0,0.0,1,3,3
1,110152,Maioni,[ Miss. Roberta],0,1,[16.0],1.0,0.0,1,3,3
2,110152,Rothes,[ the Countess. of (Lucy Noel Martha Dyer-Edwa...,0,1,[33.0],1.0,0.0,1,3,3
3,110413,Taussig,"[ Miss. Ruth, Mr. Emil, Mrs. Emil (Tillie Ma...",1,2,"[18.0, 52.0, 39.0]",3.0,0.0,3,1,3
4,110465,Clifford,[ Mr. George Quincy],1,0,[nan],0.0,0.0,1,2,2
